# Reinforce  Monte Carlo Algorithm
In this notebook, you'll code The Reinforce (also called Monte Carlo Policy Gradient) algorithm  from scratch: .

Reinforce is a *Policy-based method*: a Deep Reinforcement Learning algorithm that tries **to optimize the policy directly without using an action-value function**.

More precisely, Reinforce is a *Policy-gradient method*, a subclass of *Policy-based methods* that aims **to optimize the policy directly by estimating the weights of the optimal policy using gradient ascent**.

To test its robustness, we're going to train it in Cartpole-v1 environment

## Import the packages

In [ ]:
import numpy as np
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Gym
import gym


## Check if we have a GPU
- Let's check if we have a GPU `device:cuda0`

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Agent: Playing CartPole-v1 🤖
### The CartPole-v1 environment

> A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

So, we start with CartPole-v1. The goal is to push the cart left or right **so that the pole stays in the equilibrium.**

The episode ends if:
- The pole Angle is greater than ±12°
- Cart Position is greater than ±2.4
- Episode length is greater than 500

We get a reward 💰 of +1 every timestep the Pole stays in the equilibrium.

In [ ]:
env_id = "CartPole-v1"
# Create the env
env = gym.make(env_id)

# Create the evaluation env
eval_env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape[0]
a_size = env.action_space.n

In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

## Let's build the Reinforce Architecture
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit6/reinforce.png" alt="Reinforce"/>

So we want:
- Two fully connected layers (fc1 and fc2).
- Using ReLU as activation function of fc1
- Using Softmax to output a probability distribution over actions

In [ ]:
class Policy(nn.Module):
    def __init__(self, s_size, a_size, h_size):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return #------add code here--------#

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = #------add code here--------#
        m = Categorical(probs)
        action = #------add code here--------#
        return action.item(), m.log_prob(action)

- Since **we want to sample an action from the probability distribution over actions (i.e., Stochastic policy)**, we can't use `action = np.argmax(m)` since it will always output the action that have the highest probability.

- We need to replace with `action = m.sample()` that will sample an action from the probability distribution P(.|s)

### Let's build the Reinforce Training Algorithm
This is the Reinforce algorithm pseudocode:
<img src="https://github.com/malkiAbdelhamid/Advanced-Deep-Learning-2023-2024-esisba/blob/master/lab3_Policy_Gradient/Reinforce_algo.png?raw=true" alt="Policy gradient pseudocode"/>

# Building the parts of our algorithm #
The main steps for building a Reinforce Algo are:
1. Generates a trajectory (Line 3)
2. Compute the discounted returns (Lines 4 & 5)
3. Standardization of the returns
4. Calculate current loss (objective function $J(\theta)$) and estimate the gradient (Lines 6 & 7)
5. Update parameters (Line 8)



### Line 3 of pseudocode
Based on the policy, the method **generate_trajectory(policy, max_t)** generates a trajectory and return two lists:
  - saved_log_probs: the log probability of each action in the trajectory
  - rewards: the list of the obtained rewards

In [ ]:
 def generate_trajectory(policy, max_t):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        for t in range(max_t):
            action, log_prob = #add code here
            # add the obtained probability to saved_log_probs list
            # apply the action on the env
            # add the obtained rewards to rewards list
            if done:
                break


        return  saved_log_probs, rewards

 ### Lines 4 & 5 of pseudocode
 the method **computer_cumulative_reward(rewards,max_t,gamma)** Compute the discounted returns at each timestep, as the sum of the gamma-discounted return at time t (G_t) + the reward at time t

In [ ]:
# Line 6 of pseudocode: calculate the return

def computer_cumulative_reward(rewards,max_t,gamma):
        ##we calculate the returns at timestep t as:
        #           G_(t)= reward[t] + gamma * G_(t+1)
        #
        ## We compute this starting from the last timestep to the first, in order
        ## to employ the formula presented above and avoid redundant computations that would be needed
        ## if we were to do it from first to last.

        ## Hence, the queue "returns" will hold the returns in chronological order, from t=0 to t=n_steps
        ## thanks to the appendleft() function which allows to append to the position 0 in constant time O(1)
        ## a normal python list would instead require O(N) to do this.
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        for t in range(n_steps)[::-1]:
            disc_return_t = (returns[0] if len(returns)>0 else 0)
            returns.appendleft( #------add code here--------#)
        return returns

### standardization of the returns
- is employed to make training more stable


In [ ]:
def returns_standardization(returns):
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is
        # added to the standard deviation of the returns to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        return returns

 ### Line 6, 7 & 8 of pseudocode
**why do we minimize the loss**? You talked about Gradient Ascent not Gradient Descent?

- We want to maximize our utility function $J(\theta)$ but in PyTorch like in Tensorflow it's better to **minimize an objective function.**
    - So let's say we want to reinforce action 3 at a certain timestep. Before training this action P is 0.25.
    - So we want to modify $\theta$ such that $\pi_\theta(a_3|s; \theta) > 0.25$
    - Because all P must sum to 1, max $\pi_\theta(a_3|s; \theta)$ will **minimize other action probability.**
    - So we should tell PyTorch **to min $1 - \pi_\theta(a_3|s; \theta)$.**
    - This loss function approaches 0 as $\pi_\theta(a_3|s; \theta)$ nears 1.
    - So we are encouraging the gradient to max $\pi_\theta(a_3|s; \theta)$

In [ ]:

def compute_loss_and_train_policy(optimizer, saved_log_probs, returns):
        # Line 6
        policy_loss = []
        for log_prob, disc_return in zip(saved_log_probs, returns):
            #------add code here--------#
        policy_loss = torch.cat(policy_loss).sum()

        # Line 7 & 8: PyTorch prefers gradient descent
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()


## Merge all functions into the reinforce method ##

You will now see how the overall **Reinforce Algo** is structured by putting together all the building blocks (functions implemented in the previous parts) together, in the right order.

In [ ]:
def reinforce(policy, optimizer, n_training_episodes, max_t, gamma, print_every):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []

    for i_episode in range(1, n_training_episodes+1):

        # Line 3 of pseudocode
        saved_log_probs, rewards= #------add code here--------#
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # Lines 4 & 5 of pseudocode: calculate the return
        returns=#------add code here--------#

        ## standardization of the returns is employed to make training more stable
        returns= #------add code here--------#

        # Lines 6, 7 & 8
        #------add code here--------#


        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores

##  Train it
- We're now ready to train our agent.
- But first, we define a variable containing all the training hyperparameters.
- You can change the training parameters (and should 😉)

In [ ]:
cartpole_hyperparameters = {
    "h_size": 64,
    "n_training_episodes": 1500,
    "n_evaluation_episodes": 10,
    "max_t": 1000,
    "gamma": 1.0,
    "lr": 1e-2,
    "env_id": env_id,
    "state_space": s_size,
    "action_space": a_size,
}

In [ ]:
# Create policy and place it to the device
cartpole_policy = Policy(cartpole_hyperparameters["state_space"], cartpole_hyperparameters["action_space"], cartpole_hyperparameters["h_size"]).to(device)
cartpole_optimizer = optim.Adam(cartpole_policy.parameters(), lr=cartpole_hyperparameters["lr"])

In [ ]:
scores = reinforce(cartpole_policy,
                   cartpole_optimizer,
                   cartpole_hyperparameters["n_training_episodes"],
                   cartpole_hyperparameters["max_t"],
                   cartpole_hyperparameters["gamma"],
                   100)

In [ ]:
scores= pd.Series(scores, name="scores_REINFORCE")
scores.describe()

In [ ]:
fig, ax = plt.subplots(1, 1)
_ = scores.plot(ax=ax, label="scores_REINFORCE")
_ = (scores.rolling(window=100)
           .mean()
           .rename("Rolling Average")
           .plot(ax=ax))
ax.legend()
_ = ax.set_xlabel("Episode Number")
_ = ax.set_ylabel("scores_REINFORCE")